In [2]:
# Install Ultralytics (YOLOv8)
!pip install ultralytics roboflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 119.8 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11


In [1]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0))


CUDA available: True
GPU Name: Tesla T4


In [4]:
from roboflow import Roboflow

rf = Roboflow(api_key="Xr18Ag9t4yjLNhm0tJvN")

project = rf.workspace("cxrdataset").project("skin-ga5ww")
dataset = project.version(1).download("folder")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to skin-1 in folder:: 100%|██████████| 12261/12261 [00:02<00:00, 5386.07it/s]


In [5]:
import os

base_path = dataset.location

for split in ["train", "valid", "test"]:
    split_path = os.path.join(base_path, split)
    print(f"\n{split.upper()} CLASSES:")
    print(os.listdir(split_path))



TRAIN CLASSES:
['Basal Cell Carcinoma', 'Melanoma']

VALID CLASSES:
['Basal Cell Carcinoma', 'Melanoma']

TEST CLASSES:
['Basal Cell Carcinoma', 'Melanoma']


In [6]:
from collections import Counter

def count_images(split):
    counter = Counter()
    split_path = os.path.join(base_path, split)
    for cls in os.listdir(split_path):
        cls_path = os.path.join(split_path, cls)
        counter[cls] = len(os.listdir(cls_path))
    return counter

print("Train Distribution:", count_images("train"))
print("Validation Distribution:", count_images("valid"))
print("Test Distribution:", count_images("test"))


Train Distribution: Counter({'Melanoma': 5070, 'Basal Cell Carcinoma': 5018})
Validation Distribution: Counter({'Melanoma': 725, 'Basal Cell Carcinoma': 717})
Test Distribution: Counter({'Melanoma': 362, 'Basal Cell Carcinoma': 358})


In [8]:
from ultralytics import YOLO

# Load pretrained YOLOv8 classification model
model = YOLO("yolov8n-cls.pt")

# Train the model
model.train(
    data=base_path,
    epochs=10,
    imgsz=224,
    batch=32,
    optimizer="Adam",
    lr0=1e-3
)


Ultralytics 8.3.246 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/skin-1, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=Adam, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pr

ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f022c009130>
curves: []
curves_results: []
fitness: 0.989251047372818
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.978502094745636, 'metrics/accuracy_top5': 1.0, 'fitness': 0.989251047372818}
save_dir: PosixPath('/content/runs/classify/train2')
speed: {'preprocess': 0.08620968724033158, 'inference': 0.35072490291266695, 'loss': 0.00017362413318008845, 'postprocess': 0.0003059785013431637}
task: 'classify'
top1: 0.978502094745636
top5: 1.0

In [9]:
model.val()


Ultralytics 8.3.246 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n-cls summary (fused): 30 layers, 1,437,442 parameters, 0 gradients, 3.3 GFLOPs
train: /content/skin-1/train... found 10088 images in 2 classes ✅ 
val: /content/skin-1/valid... found 1442 images in 2 classes ✅ 
test: /content/skin-1/test... found 720 images in 2 classes ✅ 
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1478.6±581.5 MB/s, size: 36.4 KB)
val: Scanning /content/skin-1/valid... 1442 images, 0 corrupt: 100% ━━━━━━━━━━━━ 1442/1442 3.2Mit/s 0.0s
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 91/91 10.0it/s 9.1s
                   all      0.979          1
Speed: 0.1ms preprocess, 0.9ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /content/runs/classify/val


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f02d21cb7a0>
curves: []
curves_results: []
fitness: 0.989251047372818
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.978502094745636, 'metrics/accuracy_top5': 1.0, 'fitness': 0.989251047372818}
save_dir: PosixPath('/content/runs/classify/val')
speed: {'preprocess': 0.1351616809985499, 'inference': 0.9409789216366857, 'loss': 0.0035287094318097066, 'postprocess': 0.004587057559668345}
task: 'classify'
top1: 0.978502094745636
top5: 1.0

In [11]:
model.predict(
    source="/content/skin-1/test/*/*",
    save=True
)



image 1/720 /content/skin-1/test/Basal Cell Carcinoma/ISIC_0024360_jpg.rf.1966900a69862517b5a8e3475149d7d3.jpg: 224x224 Basal Cell Carcinoma 1.00, Melanoma 0.00, 2.5ms
image 2/720 /content/skin-1/test/Basal Cell Carcinoma/ISIC_0024931_jpg.rf.b5825bb65c5dcb3a3d93761df5ff82db.jpg: 224x224 Basal Cell Carcinoma 0.55, Melanoma 0.45, 2.4ms
image 3/720 /content/skin-1/test/Basal Cell Carcinoma/ISIC_0025031_jpg.rf.762a1687dd819355474660f35ee182f3.jpg: 224x224 Basal Cell Carcinoma 1.00, Melanoma 0.00, 2.4ms
image 4/720 /content/skin-1/test/Basal Cell Carcinoma/ISIC_0025691_jpg.rf.00626b4c99e0807d97a341e3227c1609.jpg: 224x224 Basal Cell Carcinoma 0.99, Melanoma 0.01, 2.2ms
image 5/720 /content/skin-1/test/Basal Cell Carcinoma/ISIC_0025700_jpg.rf.c7e8bb64f936993a814193a017af61a9.jpg: 224x224 Basal Cell Carcinoma 0.98, Melanoma 0.02, 2.3ms
image 6/720 /content/skin-1/test/Basal Cell Carcinoma/ISIC_0025826_jpg.rf.a7aedda75e35a382fd9a715f76c5be60.jpg: 224x224 Basal Cell Carcinoma 1.00, Melanoma 0.0

[ultralytics.engine.results.Results object with attributes:
 
 boxes: None
 keypoints: None
 masks: None
 names: {0: 'Basal Cell Carcinoma', 1: 'Melanoma'}
 obb: None
 orig_img: array([[[36, 35, 55],
         [36, 35, 55],
         [35, 34, 54],
         ...,
         [32, 25, 38],
         [30, 23, 36],
         [28, 21, 34]],
 
        [[36, 35, 55],
         [36, 35, 55],
         [35, 34, 54],
         ...,
         [33, 26, 39],
         [31, 24, 37],
         [29, 22, 35]],
 
        [[36, 35, 55],
         [36, 35, 55],
         [35, 34, 54],
         ...,
         [33, 26, 39],
         [31, 24, 37],
         [29, 22, 35]],
 
        ...,
 
        [[51, 51, 75],
         [52, 52, 76],
         [54, 54, 78],
         ...,
         [26, 20, 31],
         [26, 20, 31],
         [25, 19, 30]],
 
        [[50, 50, 74],
         [51, 51, 75],
         [52, 52, 76],
         ...,
         [26, 20, 31],
         [25, 19, 30],
         [25, 19, 30]],
 
        [[50, 50, 74],
         [